In [1]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pydub
from pydub.playback import play
import IPython.display as ipd
import os
output_notebook()

Loading BokehJS ...

In [115]:
# This class defines the core Guitar Effects object. 
# It contains functions to read and write audio files.
# It also contains all the different functions implementing various guitar effects

class GEcore():
    
    def __init__(self):
        self.effectname = ''
        self.audiofilename = ''
        self.framerate = []
        self.signal = []
        self.read_audiofile()
        
    def read_audiofile(self):
        name = input('Enter the audio filename you want to read including the extension: ')
        filename, file_ext = os.path.splitext(name)
        filename = os.getcwd() + '/audiofiles/' + name
        self.audiofilename = filename
        audiofile = pydub.AudioSegment.from_file(filename, file_ext)
        audiofile = audiofile.fade_out(2000)
        self.framerate = audiofile.frame_rate
        songdata = []  # Empty list for holding audio data
        channels = []  # Empty list to hold data from separate channels
        songdata = np.frombuffer(audiofile._data, np.int16)
        for chn in range(audiofile.channels):
            channels.append(songdata[chn::audiofile.channels])  # separate signal from channels
        self.signal = np.sum(channels, axis=0) / len(channels)  # Averaging signal over all channels
        p1 = self.plot_signal(self.signal)
        show(p1)
        
    def plot_signal(self, asignal):
        p = figure(plot_width=900, plot_height=500, title='Audio Signal', 
            x_axis_label='Time (s)', y_axis_label='Amplitude (arb. units)')
        time = np.linspace(0, len(asignal)/self.framerate, len(asignal))
        p.line(time[0::90], asignal[0::90])
        return(p)
    
    def delay(self, input_signal):
        delaytime = int(input('Enter the delay you want to add in miliseconds: '))
        gain = float(input('Enter the gain (a number betweeen 0 and 1): '))
        num = int(delaytime * 1e-3 * self.framerate)
        delaysig = np.roll(input_signal, num)
        delaysig[:num] = 0
        output_signal = input_signal + gain * delaysig
        p2 = self.plot_signal(output_signal)
        show(p2)
        return output_signal

In [116]:
ge = GEcore()
fig1 = ge.plot_signal(ge.signal)
ipd.Audio(ge.signal, rate=ge.framerate)

Enter the audio filename you want to read including the extension: flute.wav


In [110]:
sound_delay = ge.delay(ge.signal)
ipd.Audio(sound_delay, rate=ge.framerate)

Enter the delay you want to add in miliseconds: 1000
Enter the gain (a number betweeen 0 and 1): 0.8


In [105]:
test1 = np.array([1, 2, 3, 4, 5])
test2 = np.roll(test1, -2)
test2

array([3, 4, 5, 1, 2])